# A full business solution

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [3]:
# Import necessary libraries
import os
import requests
import json
from typing import List, Dict
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from urllib.parse import urljoin

# Load environment variables from a .env file
load_dotenv()

# Define constants
MODEL = 'gpt-4o-mini'  # Specify the OpenAI model to use
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')  # Get API key from environment or use default

# Initialize OpenAI client with the API key
openai = OpenAI(api_key=OPENAI_API_KEY)

class Website:
    """
    A class to represent a website and its contents.
    """
    def __init__(self, url: str):
        """
        Initialize the Website object with a given URL.
        
        :param url: The URL of the website to scrape
        """
        self.url = url
        self.title, self.text, self.links = self._scrape_website()

    def _scrape_website(self) -> tuple:
        """
        Scrape the website content, extracting title, text, and links.
        
        :return: A tuple containing the title, text content, and links of the website
        """
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract title
        title = soup.title.string if soup.title else "No title found"
        
        # Extract text content
        if soup.body:
            for tag in soup.body(["script", "style", "img", "input"]):
                tag.decompose()  # Remove unwanted tags
            text = soup.body.get_text(separator="\n", strip=True)
        else:
            text = ""
        
        # Extract links
        links = [link.get('href') for link in soup.find_all('a') if link.get('href')]
        
        return title, text, links

    def get_contents(self) -> str:
        """
        Get a formatted string of the website contents.
        
        :return: A string containing the website title and text content
        """
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

class LinkAnalyzer:
    """
    A class to analyze and categorize links from a website.
    """
    # System prompt for the OpenAI model to categorize links
    LINK_SYSTEM_PROMPT = """
    You are provided with a list of links found on a webpage. Your task is to first categorize each link into one of the following categories:
    - about page
    - careers page
    - terms of service
    - privacy policy
    - contact page
    - other (please specify).

    Once the links are categorized, please choose which links are most relevant to include in a brochure about the company. 
    The brochure should only include links such as About pages, Careers pages, or Company Overview pages. Exclude any links related to Terms of Service, Privacy Policy, or email addresses.

    Respond in the following JSON format:
    {
        "categorized_links": [
            {"category": "about page", "url": "https://full.url/about"},
            {"category": "careers page", "url": "https://full.url/careers"},
            {"category": "terms of service", "url": "https://full.url/terms"},
            {"category": "privacy policy", "url": "https://full.url/privacy"},
            {"category": "other", "specify": "contact page", "url": "https://full.url/contact"}
        ],
        "brochure_links": [
            {"type": "about page", "url": "https://full.url/about"},
            {"type": "careers page", "url": "https://full.url/careers"}
        ]
    }

    Please find the links below and proceed with the task:

    Links (some may be relative links):
    [INSERT LINK LIST HERE]
    """

    @staticmethod
    def get_links(website: Website) -> Dict:
        """
        Analyze and categorize links from a given website.
        
        :param website: A Website object containing the links to analyze
        :return: A dictionary containing categorized links and brochure-relevant links
        """
        # Prepare the user prompt for the OpenAI model
        user_prompt = f"Here is the list of links on the website of {website.url} - "
        user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
    Do not include Terms of Service, Privacy, email links.\n"
        user_prompt += "Links (some might be relative links):\n"
        user_prompt += "\n".join(website.links)

        # Make an API call to OpenAI for link analysis
        completion = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": LinkAnalyzer.LINK_SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(completion.choices[0].message.content)

class BrochureGenerator:
    """
    A class to generate a company brochure based on website content.
    """
    # System prompt for the OpenAI model to generate the brochure
    SYSTEM_PROMPT = """
    You are an assistant that analyzes the contents of several relevant pages from a company website 
    and creates a brochure about the company for prospective customers, investors and recruits. Respond in markdown.
    Include details of company culture, customers and careers/jobs if you have the information.
    Structure the brochure to include specific sections as follows:
    About Us
    What we do
    How We Do It
    Where We Do It
    Our People
    Our Culture
    Connect with Us.
    Please provide two versions of the brochure, the first in English, the second in Spanish. The contents of the brochure are to be the same for both languages.
    """

    @staticmethod
    def get_all_details(url: str) -> str:
        """
        Gather all relevant details from a company's website.
        
        :param url: The URL of the company's main page
        :return: A string containing all relevant website content
        """
        result = "Landing page:\n"
        website = Website(url)
        result += website.get_contents()

        # Analyze links and get brochure-relevant ones
        links = LinkAnalyzer.get_links(website)
        brochure_links = links.get('brochure_links', [])
        print("Found Brochure links:", brochure_links)

        # Gather content from brochure-relevant pages
        for link in brochure_links:
            result += f"\n\n{link['type']}:\n"
            full_url = urljoin(url, link["url"])
            result += Website(full_url).get_contents()

        return result

    @staticmethod
    def get_brochure_user_prompt(company_name: str, url: str) -> str:
        """
        Generate a user prompt for the OpenAI model to create a brochure.
        
        :param company_name: The name of the company
        :param url: The URL of the company's main page
        :return: A string containing the user prompt for brochure generation
        """
        user_prompt = f"You are looking at a company called: {company_name}\n"
        user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
        user_prompt += BrochureGenerator.get_all_details(url)
        return user_prompt[:20_000]  # Truncate if more than 20,000 characters

    @staticmethod
    def stream_brochure(company_name: str, url: str):
        """
        Generate and stream a company brochure.
        
        :param company_name: The name of the company
        :param url: The URL of the company's main page
        """
        # Make a streaming API call to OpenAI for brochure generation
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": BrochureGenerator.SYSTEM_PROMPT},
                {"role": "user", "content": BrochureGenerator.get_brochure_user_prompt(company_name, url)}
            ],
            stream=True
        )

        # Display the generated brochure in real-time
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```", "").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)


In [5]:
# Main execution block
if __name__ == "__main__":
    # Generate a brochure
    BrochureGenerator.stream_brochure("Edward Donner", "https://edwarddonner.com/")

Found Brochure links: [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}, {'type': 'other', 'specify': 'outsourcing', 'url': 'https://edwarddonner.com/outsmart/'}]



# Edward Donner Company Brochure

## About Us
Edward Donner is the creative brain behind Nebula.io, where we leverage Generative AI and advanced machine learning technologies to help recruiters effectively source, understand, engage, and manage talent. Born from a rich history in the AI landscape, our goal is simple yet profound: to aid individuals in discovering their true potential and pursuing their ikigai — their reason for being.

## What We Do
At Edward Donner, we specialize in an array of tools and services, primarily focusing on a patented matching model that connects people with roles they are optimally suited for — all without the need for keyword searches. Our platform is designed to ensure you find your dream job while having a fulfilling and engaging work experience.

## How We Do It
We employ groundbreaking, proprietary Large Language Models (LLMs) that are finely tuned to the recruitment industry. Our innovative approach is geared towards real-world application, minimizing the gap between candidates and their ideal roles. By focusing on individual strengths and needs, we drive efficiency and happiness in job placements.

## Where We Do It
Our operations orbit around the vibrant backdrop of New York City, an epicenter for talent and innovation. We create an inclusive remote work environment that thrives on collaboration, creativity, and technology, ensuring that our team and our customers can engage seamlessly, wherever they are.

## Our People
Our diverse team consists of experts in software engineering, data science, and technology leadership. Our founder, Ed, brings extensive experience and a love for programming, music, and enthusiastic problem-solving. Each individual contributes unique skills while sharing a passion for harnessing AI to tackle meaningful challenges.

## Our Culture
At Edward Donner, we pride ourselves on fostering a culture of innovation and collaboration. We aim to create a workspace that inspires creativity, encourages continuous learning, and celebrates the successes of our employees. Our mission to elevate human potential extends to our work culture, where every voice and idea is valued.

## Connect with Us
We would love to hear from you! To stay connected and explore opportunities, reach out via:
- Email: ed [at] edwarddonner [dot] com
- [Our Website](http://www.edwarddonner.com)
- Follow us on social media: [LinkedIn](#), [Twitter](#), [Facebook](#)

---

# Folleto de la Empresa Edward Donner

## Sobre Nosotros
Edward Donner es la mente creativa detrás de Nebula.io, donde aprovechamos la IA generativa y tecnologías avanzadas de aprendizaje automático para ayudar a los reclutadores a identificar, comprender, comprometer y gestionar talentos. Nacido de una rica historia en el ámbito de IA, nuestro objetivo es simple pero profundo: ayudar a las personas a descubrir su verdadero potencial y perseguir su ikigai, su razón de ser.

## Lo Que Hacemos
En Edward Donner, nos especializamos en una variedad de herramientas y servicios, centrados principalmente en un modelo de coincidencia patentado que conecta a las personas con los roles para los que están óptimamente calificadas, todo esto sin necesidad de búsquedas por palabras clave. Nuestra plataforma está diseñada para garantizar que encuentres tu trabajo soñado mientras vives una experiencia laboral satisfactoria y atractiva.

## Cómo Lo Hacemos
Empleamos modelos de lenguaje de gran tamaño (LLMs) patentados y orientados específicamente a la industria del reclutamiento. Nuestro enfoque innovador está dirigido a la aplicación del mundo real, minimizando la brecha entre los candidatos y sus roles ideales. Al centrarnos en las fortalezas y necesidades individuales, impulsamos la eficiencia y la felicidad en las colocaciones laborales.

## Dónde Lo Hacemos
Nuestras operaciones giran en torno al vibrante telón de fondo de la ciudad de Nueva York, un epicentro de talento e innovación. Creamos un entorno de trabajo remoto inclusivo que prospera en la colaboración, la creatividad y la tecnología, asegurando que nuestro equipo y nuestros clientes puedan interactuar de manera fluida, donde sea que se encuentren.

## Nuestra Gente
Nuestro diverso equipo está compuesto por expertos en ingeniería de software, ciencia de datos y liderazgo tecnológico. Nuestro fundador, Ed, aporta una amplia experiencia y un amor por la programación, la música y la resolución entusiasta de problemas. Cada individuo contribuye con habilidades únicas mientras comparte la pasión por aprovechar la IA para abordar desafíos significativos.

## Nuestra Cultura
En Edward Donner, nos enorgullece fomentar una cultura de innovación y colaboración. Nuestro objetivo es crear un espacio de trabajo que inspire la creatividad, fomente el aprendizaje continuo y celebre los éxitos de nuestros empleados. Nuestra misión de elevar el potencial humano se extiende a nuestra cultura laboral, donde cada voz e idea es valorada.

## Conéctate Con Nosotros
¡Nos encantaría saber de ti! Para mantener la conexión y explorar oportunidades, contáctanos a través de:
- Email: ed [at] edwarddonner [dot] com
- [Nuestro Sitio Web](http://www.edwarddonner.com)
- Síguenos en redes sociales: [LinkedIn](#), [Twitter](#), [Facebook](#)

